# Russia-Ukraine Conflict Analysis

The following notebook works as a work of research for the analysis of data from the Ethereum network during the period of the Russian-Ukraine conflict.


## Import Statements

In [1]:
import pandas as pd
import numpy as np
from enum import Enum
import seaborn as sns

In [2]:
class DegreeType(Enum):
    IN_DEGREE = 1
    OUT_DEGREE = 2
    DEGREE = 3

class ETHDegreeAnalyzer:
    '''
    Class responsible for handling degree comparasions in Ethereum for different time periods.

    ----------

    #### Attributes:
    
    - datasets_name: str

            Name used for the filenames of the lookup datasets. Defaults to "lookup".

    - lookup_path: str
    
            Path for the lookup dataset files that will be used for the comparasions. Defaults to the current working directory.
    - degree_type: DegreeType

            Type of degree analysis to be performed. Acceps DegreeType.IN_DEGREE, DegreeType.OUT_DEGREE and DegreeType.DEGREE. 
            Defaults to DegreeType.DEGREE.

    '''
    def __init__(
            self,
            datasets_name: str = 'lookup',
            lookup_path: str = './',
            degree_type: DegreeType = DegreeType.DEGREE
        ):

        self.datasets_name = datasets_name
        self.lookup_path = lookup_path
        self.degree_type = degree_type

    
    def _concat_lookup(
            self,
            df: pd.DataFrame,
            df_id: str,
            suffix: str
        ) -> pd.DataFrame:
        
        try:  
            df_name = self.lookup_path+self.datasets_name+'_'+df_id+suffix+'.csv'
            new_df = pd.read_csv(df_name, index_col = 0)
            new_df = pd.DataFrame(new_df.mean(axis = 1), columns = ["avgValue"+df_id])
        except Exception as e:
            print(f'Warning: {df_name} could not be read": {e}')
            return df
        return df.merge(new_df, how='left', left_index=True, right_index=True).fillna(0)

    def load_dataset(self) -> None:
        '''
        Loads into memory all the datasets from the filepath specified concatenated. If any is missing, it will be ignored and
        the function will try to concatenate the others.
        '''
        match(self.degree_type):
            case DegreeType.IN_DEGREE:
                suffix = '_in'
            case DegreeType.OUT_DEGREE:
                suffix = '_out'
            case DegreeType.DEGREE:
                suffix = ''
            case _:
                raise ValueError('Degree type not recognized.')

        self._df = (
            pd.read_csv(self.lookup_path+self.datasets_name+'_war'+suffix+'.csv', index_col = 0)
            .pipe(self._concat_lookup, df_id = '1', suffix = suffix)
            .pipe(self._concat_lookup, df_id = '2', suffix = suffix)
            .pipe(self._concat_lookup, df_id = '3', suffix = suffix)
            .pipe(self._concat_lookup, df_id = '4', suffix = suffix)
        )

    

In [3]:
degree_analyzer = ETHDegreeAnalyzer(lookup_path='../data/')

In [4]:
degree_analyzer.load_dataset()

In [6]:
degree_analyzer._df

,degree,avgValue1,avgValue2,avgValue3
0x0000000000000000000000000000000000000000,579.0,2251.333333,1893.0,1288.333333
0x0000000000000000000000000000000000000001,16.0,5.000000,5.0,76.000000
0x0000000000000000000000000000000000000064,1.0,0.000000,0.0,0.333333
0x0000000000000000000000000000000000000800,2.0,0.000000,0.0,0.000000
0x0000000000000000000000000000000000001010,7.0,1.000000,5.0,3.666667
...,...,...,...,...
0xffffffff2ba8f66d4e51811c5190992176930278,133.0,2455.666667,871.0,492.333333
0xfffffffff15abf397da76f1dcc1a1604f45126db,67.0,1921.333333,407.5,191.666667
0xffffffffff2419497bf75e415bc9a7d446e05c0f,2.0,17.333333,55.0,13.000000
0xffffffffff402b1b62421a978cf93a56453d1496,1.0,0.000000,0.0,0.000000
